In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:

import matplotlib.backend_bases
import matplotlib.pyplot as plt
import scipy.sparse
import scipy.stats as sp
from preliminary_tests import conv_jit, main
from sklearn.neighbors import KernelDensity
import numpy as np

In [ ]:
ret = main()

In [ ]:
xs = ret[0].distr()[0]
ps = ret[0].distr()[1]

In [ ]:
X = np.stack([xs, ps]).T
X.shape

In [ ]:
xs[:, np.newaxis]

In [ ]:
X_plot = np.linspace(0,2,1000)[:, np.newaxis]
gaus_dens_est = KernelDensity(kernel="gaussian", bandwidth=.1).fit(xs[:, np.newaxis], sample_weight=ps)
linear_dens_est = KernelDensity(kernel="linear", bandwidth=.1).fit(xs[:, np.newaxis], sample_weight=ps)

In [ ]:
gaus_log_dens = gaus_dens_est.score_samples(X_plot)
linear_log_dens = linear_dens_est.score_samples(X_plot)


In [ ]:
fig, ax = plt.subplots(2,2, sharex=True, sharey=True)
ax[0,0].plot(xs, ps)
ax[1, 0].fill(X_plot[:, 0], np.exp(gaus_log_dens), fc="#AAAAFF")
ax[1, 1].fill(X_plot[:, 0], np.exp(linear_log_dens), fc="#AAAAFF")
ax[1, 0].text(-3.5, 0.31, "Gaussian Kernel Density")
plt.show()

In [ ]:
from visualize import get_pseudo_density


In [ ]:
x, y = get_pseudo_density(ret[0])

In [ ]:

plt.plot(x, y)

In [ ]:
cdf = ret[0].get_cdf()

In [ ]:
cdf

In [ ]:
plt.plot(cdf[0], cdf[1])

# MDP from paper


In [ ]:
from preliminary_tests import RV_Discrete, Policy, TransitionKernel
from preliminary_tests import CategoricalRewardDistr, CategoricalDistrCollection, MDP
from preliminary_tests import categorical_dbo
from distributions import emp_normal
from typing import List
from visualize import get_pseudo_density
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sp


In [ ]:
# states, actions, transition probs, Pi, discount factor
states = [0, 1, 2]
actions = [0]
state_action_probs = {0: np.array([1.0]),
                      1: np.array([1.0]),
                      2: np.array([1.0])}

pi = Policy(states=states, actions=actions, probs=state_action_probs)
discount_factor = np.float64(0.7)

trans_probs = {(0,0): np.array([0.0, 1.0, 0.0]),
               (1,0): np.array([0.0, 0.0, 1.0]),
               (2,0): np.array([1.0, 0.0, 0.0])}

transition_kernel = TransitionKernel(states, actions, trans_probs)

In [ ]:
# reward distributions for (state, action, next_state) triples
r_001 = emp_normal(-3, 1, 10)
r_102 = emp_normal(5, 2, 10)
r_200 = emp_normal(0, 0.5, 10)

state_action_state = [(0,0,1), (1,0,2), (2,0,0)]


rewards = CategoricalRewardDistr(state_action_pairs=state_action_state,
                                 distributions=[r_001, r_102, r_200])




In [ ]:
r_001.distr()
_x, _y = get_pseudo_density(r_200)
plt.plot(_x, _y)

In [ ]:
# initial distributions for each state 
distribution_1: RV_Discrete = RV_Discrete(
    xk=np.array([-3.0, 1.0]), pk=np.array([0.5, 0.5])
)
distribution_2: RV_Discrete = RV_Discrete(
    xk=np.array([-3.0, 1.0]), pk=np.array([0.5, 0.5])
)
distribution_3: RV_Discrete = RV_Discrete(
    xk=np.array([-3.0, 1.0]), pk=np.array([0.5, 0.5])
)
distributions: List[RV_Discrete] = [distribution_1, distribution_2, distribution_3]
total_reward_distr_estimate: CategoricalDistrCollection = (
    CategoricalDistrCollection(states, distributions)
)
    

In [ ]:
mdp: MDP = MDP(states, actions, rewards, transition_kernel)
mdp.set_policy(pi)

res: CategoricalDistrCollection = total_reward_distr_estimate
for i in range(8):
    print(f"Iteration {i} started.")
    res: CategoricalDistrCollection = categorical_dbo(mdp, pi, res)
    print(f"Iteration {i} stopped.")

print(res[0].distr())


In [ ]:
from preliminary_tests import project_eqi
type(res)
# x, y = get_pseudo_density(res[0])
# plt.plot(x,y)
xs, ps = project_eqi(values=res[0].distr()[0],
            probs=res[0].distr()[1],
            no_of_bins=1000, state=0)
print(len(res[0].distr()[0]), len(res[0].distr()[1]))
print(len(xs), len(ps))

# print(len(x), len(y))



In [ ]:
_xs, _ps = get_pseudo_density(RV_Discrete(xs, ps))

In [ ]:
import scipy.stats as sp
plt.plot(_xs, _ps)
comp_x = np.linspace(-3,3,1000)
plt.plot(comp_x, sp.norm.pdf(comp_x, 0.761, np.sqrt(2.3800)))
# plt.plot(comp_x, sp.norm.pdf(comp_x, 0.761, 2.380O))

In [ ]:
temp_distr = RV_Discrete(xs, ps)
temp_distr_cdf = temp_distr.get_cdf()

plt.plot(temp_distr_cdf[0], temp_distr_cdf[1])
plt.plot(comp_x, sp.norm.cdf(comp_x, 0.761, np.sqrt(2.3800)))